# The Newey-West Covariance Matrix

The Newey-West (NW) covariance matrix of estimated regression coefficients is designed to handle both heteroskedasticity (similar to White's covariance matrix), but also autocorrelation. 

This notebook illustrates how it works.

## Load Packages

In [1]:
using Dates, DelimitedFiles, Statistics, LinearAlgebra

include("jlFiles/printmat.jl")

printyellow (generic function with 1 method)

## Load Data

In [2]:
x   = readdlm("Data/USCMRatesPs.csv",',',skipstart=1)
y   = convert(Array{Float64},x[:,3:end])   #interest rates, 3m to 10y, skip FFR
y   = y[:,[1,7]]                           #3m and 10y interest rates  

(T,n) = size(y)

(570, 2)

# Newey-West

Let $g_t$ be a vector of data series.

To calculate the Newey-West covariance matrix, we first need the
autocovariance matrices $\Omega_{s}=\text{Cov}(g_{t},g_{t-s})  $, 
which is estimated as 
$ 
\sum_{t=s+1}^{T} (g_{t}-\bar{g})(g_{t-s}-\bar{g})^{\prime}/T.
$

Then we form a linear
combination (with tent-shaped weights) of those autocovariance matrices (from
lag $-m$ to $m$), or equivalently

$
\text{Cov}(\sqrt{T}\bar{g})  = 
\Omega_{0} + \sum_{s=1}^{m}( 1-\frac{s}{m+1})  
(\Omega_{s}+\Omega_{s}^{\prime}).
$

Notice that this is the covariance matrix of $\frac{1}{\sqrt{T}}\sum_{t=1}^T g_t.$ 

Instead,if you want covariance matrix of $\frac{1}{T}\sum_{t=1}^T g_t$, then *divide* the result by $T$. 
However, if you want the covariance matrix of $\sum_{t=1}^T g_t$, then *multiply* the result by $T$.

The cells below illustrate this by estimating the std of a sample average in different ways.

Instead, if we instead need $\text{Cov}(\sqrt{T}\bar{g})$

In [3]:
"""
    NWFn(g0,m=0)

Calculates covariance matrix of sqrt(T)*sample average.

# Usage
S = NWFn(g0,m)

# Input
- `g0::Array`: Txq array of q moment conditions
- `m:int`: scalar, number of lags to use

# Output
- `S::Array`: qxq covariance matrix of sqrt(T)*sample average

"""
function NWFn(g0,m=0)

  T = size(g0,1)                    #g0 is Txq
  m = min(m,T-1)                    #number of lags

  g = g0 .- mean(g0,dims=1)         #normalizing to Eg=0
  
  S = g'g/T                         #(qxT)*(Txq)
  for s = 1:m
    Gamma_s = g[s+1:T,:]'g[1:T-s,:]/T   #same as Sum[g(t)*g(t-s)',t=s+1,T]
    S       = S  +  ( 1 - s/(m+1) ) * (Gamma_s + Gamma_s')
  end

  return S

end

NWFn

To illustrate the importance of accounting for the autocorrelations, the next cell simulates two time series with very different persistence and then calculates the traditional (assuming iid) and NW standard deviations for the sample average.

In [4]:
Cov_iid = cov(y)/T
Cov_NW  = NWFn(y,5)/T

println("Covariance matrix of average y according to an iid assumption:")
printmat(Cov_iid)

println("...and according NW")
printmat(Cov_NW)

Covariance matrix of average y according to an iid assumption:
     0.020     0.016
     0.016     0.015

...and according NW
     0.118     0.097
     0.097     0.091

